# Importing llamas_pyjamas modules

**Please follow the installation instructions prior to attempting this**

In [ ]:
import os
import sys
import numpy as np
import pickle
import ray
import pkg_resources
import glob
import traceback
from   pathlib import Path
from   config import BASE_DIR, OUTPUT_DIR, DATA_DIR

# Get package root and add to path before other imports as a precaution -> if installed as package this should hopefully not be needed
package_root = Path().absolute().parent
sys.path.append(str(package_root))
sys.path.append(BASE_DIR+'/')


ray.init(ignore_reinit_error=True)
import llamas_pyjamas.Trace.traceLlamasMulti as trace # type: ignore
import llamas_pyjamas.Extract.extractLlamas as extract # type: ignore
from llamas_pyjamas.Image.WhiteLight import WhiteLight, WhiteLightFits

In [ ]:
# Get absolute path to llamas_pyjamas package
package_path = pkg_resources.resource_filename('llamas_pyjamas', '')
package_root = os.path.dirname(package_path)

print(f"Package path: {package_path}")
print(f"Package root: {package_root}")

**Optional Ray initialisation if attempting to parallelise some of the processes**

In [ ]:
# Configure Ray runtime environment
runtime_env = {
    "py_modules": [package_root],
    "env_vars": {"PYTHONPATH": f"{package_root}:{os.environ.get('PYTHONPATH', '')}"},
    "excludes": [
        str(Path(DATA_DIR) / "**"),  # Exclude DATA_DIR and all subdirectories
        "**/*.fits",                 # Exclude all FITS files anywhere
        "**/*.gz",                 # Exclude all tarballs files anywhere
        "**/*.zip",                 # Exclude all zip files anywhere
        "**/*.pkl",                  # Exclude all pickle files anywhere
        "**/.git/**",               # Exclude git directory
    ]
}

# Initialize Ray
ray.shutdown()
ray.init(runtime_env=runtime_env)

In [ ]:
DATA_DIR = "/Users/simcoe/Science/LLAMAS/CommissioningRun/NIGHT5"
import importlib
importlib.reload(trace)

# Extraction

To run the extraction process on a single fits image and then plot a single spectrum:

In [ ]:
from llamas_pyjamas.GUI.guiExtract import GUI_extract
filepath = 'Path_to_your_fits_file'
filepath = '/Users/slh/Documents/Projects/Magellan_dev/LLAMAS/comissioning_data/20241128/LLAMAS_2024-11-28T03_50_39.584_mef.fits'
# Example: filepath = '/Users/slh/Documents/Projects/Magellan_dev/LLAMAS/comissioning_data/20241128/LLAMAS_2024-11-28T03_50_39.584_mef.fits
 
GUI_extract(filepath)


In [ ]:
#This should take a few mintues to run and in your output folder you should see the extracted pickle file
extract_pickle = 'output/your_extraction_file.pkl'
extract_pickle = '/Users/slh/Documents/Projects/Magellan_dev/LLAMAS/llamas-pyjamas/llamas_pyjamas/output/LLAMASExtract_batch_20250203_215658.pkl'
#Example: extract_pickle = '/Users/slh/Documents/Projects/Magellan_dev/LLAMAS/llamas-pyjamas/llamas_pyjamas/output/LLAMASExtract_batch_20250203_215658.pkl'

with open(extract_pickle, 'rb') as f:
    exobj = pickle.load(f)

extraction_list = exobj['extractions']

#select which extraction object you wish to plot
# Each object represents a single HDU extension in the FITS file which was extracted. 
# The order should correspond to the orider in the fits file

HDU_idx = 0 #change the '0' to the value of the HDU you wish to plot spectra from
spec_arrays = extraction_list[HDU_idx].counts #change the '0' to the value of the HDU you wish to plot spectra from

#This gives you an array of the extracted spectra for the selected HDU, shape = (Nfib, 2048)
#You can plot the spectra for a single fiber using the following code

import matplotlib.pyplot as plt
Nfib = 0 #select the fiber you wish to plot
plt.plot(spec_arrays[Nfib])
plt.show()

#To load in the extracted pickle file and plot the spectra for a single fiber

# then follow the steps from the exobjs line onwards


To run the extraction on a single HDU extension

In [ ]:
from llamas_pyjamas.Extract.extractLlamas import ExtractLlamas
import pickle
from astropy.io import fits

### MAKE SURE THE COLOUR AND BENCHSIDE OF THE TRACEFILE YOU LOAD IN MATCHES THE HDU YOU WANT TO EXTRACT
#e.g. red_1A typically corresponds to hdu[1] in the fits file
#To check this print hdu[i].header
with open(DATA_DIR + '/Your_trace_file.pkl', 'rb') as f:
    traceobj = pickle.load(f)

hdu = fits.open('Path_to_your_fits_file.fits')
ex = ExtractLlamas(traceobj, hdu[hdu_index].data.astype(float), dict(hdu[hdu_index].header))

## Extract the flat field (for getting fiber throughputs later)

In [ ]:
skyflat_filename = filename
trace_filename = os.path.join(DATA_DIR, 'LLAMAS_2024-11-29T23_50_11.041_mef.fits')
extract.ExtractLlamasCube(skyflat_filename, trace_filename)

## Extract an arc frame

In [ ]:
importlib.reload(extract)
arc_filename   = os.path.join(DATA_DIR, 'LLAMAS_2024-11-29T23_07_53.063_mef.fits')
extract.ExtractLlamasCube(arc_filename, trace_filename)

### Read in and plot up arc extractions

In [ ]:
arc_picklename = os.path.join(OUTPUT_DIR, os.path.basename(arc_filename).replace('_mef.fits', '_extract.pkl'))
arcspec, metadata = extract.load_extractions(arc_picklename)

In [ ]:
import llamas_pyjamas.Arc.arcLlamas as arc
importlib.reload(arc)
arc.shiftArcX(arc_picklename)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib qt
shift_picklename = arc_picklename.replace('_extract.pkl', '_extract_shifted.pkl')
arcspec_shifted, metadata_shifted = extract.load_extractions(shift_picklename)
[plt.plot(arcspec_shifted[18].xshift[i,:], arcspec_shifted[18].counts[i,:],".") for i in range(100,200)]
plt.show()

In [ ]:
dir(arcspec_shifted[0])

In [ ]:
import matplotlib.pyplot as plt
print(metadata)
plt.plot(arcspec[12].counts[150])
plt.plot(arcspec[0].counts[150])
plt.show()

In [ ]:
from pypeit.core.wavecal.wvutils import xcorr_shift_stretch
func = 'quadratic'
fiber = 200
success, shift, stretch, stretch2, _, _, _ = xcorr_shift_stretch(arcspec[1].counts[fiber], arcspec[7].counts[150], stretch_func=func)
print(success, shift, stretch, stretch2)
x = np.arange(2048)
plt.plot((x*stretch+x**2*stretch2)+shift, arcspec[7].counts[150])
plt.plot(x, arcspec[1].counts[fiber])

plt.show()

In [ ]:
arcspec[10].trace.fiberimg
